In [26]:
import sys
import os
import pandas as pd
import numpy as np

import pickle
from sklearn import preprocessing, metrics
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from torch import std
import os, pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn import metrics, ensemble

In [7]:
def label_class(row, option):
        
    if option == 'PETandDX':
        if row['AV45+'] == 1 and row['DX'] == 'Dementia':
            return 1
        if row['AV45+'] == 0 and row['DX'] == 'CN':
            return 0
        if np.isnan(row['AV45+']):
            if row['PIB+'] == 1 and row['DX'] == 'Dementia':
                return 1
            if row['PIB+'] == 0 and row['DX'] == 'CN':
                return 0
    
    if option == 'PET':
        if row['AV45+'] == 1:
            return 1
        if row['AV45+'] == 0:
            return 0
        if np.isnan(row['AV45+']):
            if row['PIB+'] == 1:
                return 1
            if row['PIB+'] == 0:
                return 0
    
    if option == 'LOAD':
        if row['Phenotype'] == 2:
            return 1
        if row['Phenotype'] == 1:
            return 0
        
def create_class_ADNI(df, option):

    
    df['y'] = df.apply (lambda row: label_class(row, option), axis=1)

    to_drop = list(df.columns)[-27:-1]
    df.drop(columns=to_drop, inplace=True)
    df_notna = df[df['y'].notna()]

    # print('Class distribution:')
    # print(df['y'].value_counts())

    return df_notna

In [10]:
data = pd.read_csv('data/table_datasets/AD_PPI_missense_ADNI_labeled.csv', index_col = 0)
data_wclass = create_class_ADNI(data, 'PET')
data_wclass.loc[:, (data_wclass != 0).any(axis=0)]

,APOE,BAX,CHRNA7,ENO1,GSK3B,IGF1,INSR,PICALM,PSEN1,SOD2,y
003_S_1057,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
003_S_0908,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
003_S_1122,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
136_S_0873,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
130_S_0886,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
023_S_4501,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
070_S_4692,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
005_S_4707,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
003_S_4350,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
x = data_wclass.drop(columns=['y'])
x = x['APOE']

y = data_wclass['y']
x.index = x.index.str.upper()

In [40]:
results = []
for i in range(10):

    i += 1

    f = open(f'data/splits/10Fold_CV_PET/k{i}_PET.pkl', 'rb')
    split = pickle.load(f)
    f.close()

    tr_idx = split['train']
    te_idx = split['valid']

    x_train, x_test = x.iloc[tr_idx], x.iloc[te_idx]
    y_train, y_test = y[tr_idx], y[te_idx]

    x_train = np.array(x_train).reshape(-1, 1)
    x_test = np.array(x_test).reshape(-1, 1)

    param_grid = [{'n_estimators': [50, 500, 5000]}]
    rnd_clf = RandomForestClassifier()
    
    grid_search = GridSearchCV(rnd_clf, param_grid, cv=10, scoring='roc_auc', n_jobs = -1, verbose = 3)
    grid_search.fit(x_train, y_train)
    
    print(grid_search.best_estimator_)
    
    y_prob_rf = grid_search.predict_proba(x_test)
    y_pred_rf = grid_search.predict(x_test)
    print('Acc', metrics.accuracy_score(y_test, y_pred_rf))
    print('AUC', metrics.roc_auc_score(y_test, y_prob_rf[:, 1]))
    
    results.append(metrics.roc_auc_score(y_test, y_prob_rf[:, 1]))
    print()

results = np.array(results, np.float)
print(np.mean(results), np.std(results))
    
#     tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred_rf).ravel()
#     ax = plt.subplot()
#     heatmap([[tn, fp], [fn, tp]], annot=True, fmt='g', ax=ax, center=1)
#     ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
#     ax.set_title('Confusion Matrix');
#     ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

Fitting 10 folds for each of 3 candidates, totalling 30 fits
RandomForestClassifier(n_estimators=50)
Acc 0.684931506849315
AUC 0.65625

Fitting 10 folds for each of 3 candidates, totalling 30 fits
RandomForestClassifier(n_estimators=50)
Acc 0.6301369863013698
AUC 0.6215701219512195

Fitting 10 folds for each of 3 candidates, totalling 30 fits
RandomForestClassifier(n_estimators=50)
Acc 0.5616438356164384
AUC 0.5914634146341464

Fitting 10 folds for each of 3 candidates, totalling 30 fits
RandomForestClassifier(n_estimators=500)
Acc 0.6986301369863014
AUC 0.7339939024390243

Fitting 10 folds for each of 3 candidates, totalling 30 fits
RandomForestClassifier(n_estimators=5000)
Acc 0.7808219178082192
AUC 0.7660060975609756

Fitting 10 folds for each of 3 candidates, totalling 30 fits
RandomForestClassifier(n_estimators=50)
Acc 0.6164383561643836
AUC 0.6467225609756098

Fitting 10 folds for each of 3 candidates, totalling 30 fits
RandomForestClassifier(n_estimators=50)
Acc 0.58333333333333

In [41]:
results = []
for i in range(10):

    i += 1

    f = open(f'data/splits/10Fold_CV_PET/k{i}_PET.pkl', 'rb')
    split = pickle.load(f)
    f.close()

    tr_idx = split['train']
    te_idx = split['valid']

    x_train, x_test = x.iloc[tr_idx], x.iloc[te_idx]
    y_train, y_test = y[tr_idx], y[te_idx]

    x_train = np.array(x_train).reshape(-1, 1)
    x_test = np.array(x_test).reshape(-1, 1)
    
    scaler = preprocessing.StandardScaler().fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)

    param_grid = [{'penalty' : ['l1'],'alpha': [0.0001, 0.001, 0.01, 0.1, 1]}]
    pcp_clf = Perceptron()
    
    grid_search_pcp = GridSearchCV(pcp_clf, param_grid, cv=10, scoring='roc_auc', n_jobs = -1, verbose = 3)
    grid_search_pcp.fit(x_train, y_train)
    print(grid_search_pcp.best_params_)

    y_pred_perceptron = grid_search_pcp.predict(x_test_scaled)
    print('Acc', metrics.accuracy_score(y_test, y_pred_perceptron))
    print('AUC', metrics.roc_auc_score(y_test, y_pred_perceptron))
    
    results.append(metrics.roc_auc_score(y_test, y_pred_perceptron))
    print()


results = np.array(results, np.float)
print(np.mean(results), np.std(results))
    
#     tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred_rf).ravel()
#     ax = plt.subplot()
#     heatmap([[tn, fp], [fn, tp]], annot=True, fmt='g', ax=ax, center=1)
#     ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
#     ax.set_title('Confusion Matrix');
#     ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.0001, 'penalty': 'l1'}
Acc 0.4383561643835616
AUC 0.5

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.01, 'penalty': 'l1'}
Acc 0.4383561643835616
AUC 0.5

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.0001, 'penalty': 'l1'}
Acc 0.4383561643835616
AUC 0.5

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.0001, 'penalty': 'l1'}
Acc 0.4383561643835616
AUC 0.5

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.01, 'penalty': 'l1'}
Acc 0.4383561643835616
AUC 0.5

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.001, 'penalty': 'l1'}
Acc 0.6164383561643836
AUC 0.6310975609756098

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.01, 'penalty': 'l1'}
Acc 0.4444444444444444
AUC 0.5082612116443745

Fitting 10 folds for each of 5 candidates, totalling 50 fits
{'alpha': 0.01, 'pen

In [45]:
results = []
for i in range(10):

    i += 1

    f = open(f'data/splits/10Fold_CV_PET/k{i}_PET.pkl', 'rb')
    split = pickle.load(f)
    f.close()

    tr_idx = split['train']
    te_idx = split['valid']

    x_train, x_test = x.iloc[tr_idx], x.iloc[te_idx]
    y_train, y_test = y[tr_idx], y[te_idx]

    x_train = np.array(x_train).reshape(-1, 1)
    x_test = np.array(x_test).reshape(-1, 1)
    
    scaler = preprocessing.StandardScaler().fit(x_train)
    x_train_scaled = scaler.transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    
    logreg = LogisticRegression()
    logreg.fit(x_train_scaled, y_train)

    y_prob_logreg = logreg.predict_proba(x_test_scaled)
    y_pred_logreg = logreg.predict(x_test_scaled)
    print('Acc', metrics.accuracy_score(y_test, y_pred_logreg))
    print('AUC', metrics.roc_auc_score(y_test, y_prob_logreg[:, 1]))
    
    results.append(metrics.roc_auc_score(y_test, y_pred_logreg))
    print()

results = np.array(results, np.float)
print(np.mean(results), np.std(results))
    
#     tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred_rf).ravel()
#     ax = plt.subplot()
#     heatmap([[tn, fp], [fn, tp]], annot=True, fmt='g', ax=ax, center=1)
#     ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels');
#     ax.set_title('Confusion Matrix');
#     ax.xaxis.set_ticklabels(['0', '1']); ax.yaxis.set_ticklabels(['0', '1']);

Acc 0.6575342465753424
AUC 0.7278963414634146

Acc 0.5753424657534246
AUC 0.6375762195121951

Acc 0.5616438356164384
AUC 0.5457317073170732

Acc 0.547945205479452
AUC 0.6882621951219512

Acc 0.6164383561643836
AUC 0.7629573170731707

Acc 0.6164383561643836
AUC 0.6154725609756098

Acc 0.5972222222222222
AUC 0.5881195908733281

Acc 0.6666666666666666
AUC 0.6565696302124311

Acc 0.5694444444444444
AUC 0.5743509047993706

Acc 0.5833333333333334
AUC 0.6085759244689221

0.5746102970102281 0.05129279677327344
